In [1]:
import torch
import numpy as np
import pandas as pd
import lightning as L
import torch.nn as nn
import torch.nn.functional as F
import sklearn
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
import torch.utils
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import re
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt


In [2]:
if (torch.cuda.is_available()):
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
std_scaler = StandardScaler()

In [4]:
lbl_enc = LabelEncoder()

In [5]:
data = pd.read_csv('C:/Users/Arseny/Downloads/car_price.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Arseny/Downloads/car_price.csv'

In [ ]:
data.head()

,Unnamed: 0,car_name,car_prices_in_rupee,kms_driven,fuel_type,transmission,ownership,manufacture,engine,Seats
0,0,Jeep Compass 2.0 Longitude Option BSIV,10.03 Lakh,"86,226 kms",Diesel,Manual,1st Owner,2017,1956 cc,5 Seats
1,1,Renault Duster RXZ Turbo CVT,12.83 Lakh,"13,248 kms",Petrol,Automatic,1st Owner,2021,1330 cc,5 Seats
2,2,Toyota Camry 2.5 G,16.40 Lakh,"60,343 kms",Petrol,Automatic,1st Owner,2016,2494 cc,5 Seats
3,3,Honda Jazz VX CVT,7.77 Lakh,"26,696 kms",Petrol,Automatic,1st Owner,2018,1199 cc,5 Seats
4,4,Volkswagen Polo 1.2 MPI Highline,5.15 Lakh,"69,414 kms",Petrol,Manual,1st Owner,2016,1199 cc,5 Seats


In [ ]:
data.drop(columns=['Unnamed: 0', 'car_name'], inplace=True)

In [ ]:
data[['fuel_type', 'transmission']] = data[['fuel_type', 'transmission']].apply(lambda x: lbl_enc.fit_transform(x))

In [ ]:
data = pd.get_dummies(data, columns=[ 'transmission', 'Seats'], dtype=int)

In [ ]:
data

,car_prices_in_rupee,kms_driven,fuel_type,ownership,manufacture,engine,transmission_0,transmission_1,Seats_2 Seats,Seats_4 Seats,Seats_5 Seats,Seats_6 Seats,Seats_7 Seats,Seats_8 Seats
0,10.03 Lakh,"86,226 kms",1,1st Owner,2017,1956 cc,0,1,0,0,1,0,0,0
1,12.83 Lakh,"13,248 kms",4,1st Owner,2021,1330 cc,1,0,0,0,1,0,0,0
2,16.40 Lakh,"60,343 kms",4,1st Owner,2016,2494 cc,1,0,0,0,1,0,0,0
3,7.77 Lakh,"26,696 kms",4,1st Owner,2018,1199 cc,1,0,0,0,1,0,0,0
4,5.15 Lakh,"69,414 kms",4,1st Owner,2016,1199 cc,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5507,28.90 Lakh,"45,000 kms",1,1st Owner,2018,2995 cc,1,0,0,0,0,0,1,0
5508,64.90 Lakh,"29,000 kms",4,2nd Owner,2015,1968 cc,1,0,0,0,1,0,0,0
5509,13.75 Lakh,"90,000 kms",1,2nd Owner,2013,2755 cc,1,0,0,0,1,0,0,0
5510,29.90 Lakh,"79,000 kms",1,3rd Owner,2015,2967 cc,1,0,0,0,0,1,0,0


In [ ]:
def extract_number(text):
    match = re.search(r'[\d,]+\.?\d*', str(text))
    if match:
        num_str = match.group().replace(',', '')
        return float(num_str) if '.' in num_str else int(num_str)
    return None

In [ ]:
data = data.map(extract_number)

In [ ]:
data[['kms_driven','manufacture','engine']] = data[['kms_driven','manufacture','engine']].apply(lambda x: (x - x.mean()) / x.std())

In [ ]:
data

,car_prices_in_rupee,kms_driven,fuel_type,ownership,manufacture,engine,transmission_0,transmission_1,Seats_2 Seats,Seats_4 Seats,Seats_5 Seats,Seats_6 Seats,Seats_7 Seats,Seats_8 Seats
0,10.03,0.549996,1,1,0.393192,0.731513,0,1,0,0,1,0,0,0
1,12.83,-1.194048,4,1,1.411529,-0.349268,1,0,0,0,1,0,0,0
2,16.40,-0.068561,4,1,0.138608,1.660363,1,0,0,0,1,0,0,0
3,7.77,-0.872665,4,1,0.647776,-0.575438,1,0,0,0,1,0,0,0
4,5.15,0.148219,4,1,0.138608,-0.575438,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5507,28.90,-0.435232,1,1,0.647776,2.525333,1,0,0,0,0,0,1,0
5508,64.90,-0.817603,4,2,-0.115976,0.752231,1,0,0,0,1,0,0,0
5509,13.75,0.640188,1,2,-0.625145,2.110976,1,0,0,0,1,0,0,0
5510,29.90,0.377308,1,3,-0.115976,2.476991,1,0,0,0,0,1,0,0


In [ ]:
continous_cols = [0,1,2]

In [ ]:
X = data.drop(columns='car_prices_in_rupee').to_numpy()

In [ ]:
y = data['car_prices_in_rupee'].to_numpy().reshape(-1,1)

In [ ]:
X[0]

array([0.54999617, 1.        , 1.        , 0.39319213, 0.73151301,
       0.        , 1.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        ])

In [ ]:
data.isna().sum().sum()

0

In [ ]:
y[0]

array([10.03])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [ ]:
X_train[0].shape

(13,)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x, dtype= torch.float32, device=device)
        self.y = torch.tensor(y, dtype=torch.float32, device=device)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [ ]:
train_data = MyDataset(X_train, y_train)
test_data = MyDataset(X_test, y_test)

In [ ]:
train_data[0]

(tensor([ 1.5005,  1.0000,  1.0000,  0.3932, -0.5789,  0.0000,  1.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000], device='cuda:0'),
 tensor([8.1000], device='cuda:0'))

In [ ]:
batch_size = 512
testloader = DataLoader(test_data, batch_size=batch_size, shuffle = False)
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle = True)

In [ ]:
input_size = len(X_train[0])

In [ ]:
class RegressionNN(nn.Module):
    def __init__(self, input_size):
        super(RegressionNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),

            nn.ReLU(),
            nn.Dropout(0.3),
            

            nn.Linear(512, 256),

            nn.ReLU(),
            nn.Dropout(0.2),
            

            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        return self.layers(x)

In [ ]:
model = RegressionNN(input_size=input_size).to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [ ]:
epochs = 1000
for epoch in tqdm(range(epochs), desc='Training'):
    epoch_loss =0.0
    model.train()

    for input, target in trainloader:
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()
    
    avg_loss = epoch_loss/len(trainloader)

    tqdm.write(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1/1000 | Loss: 1659.4504
Epoch 2/1000 | Loss: 1689.9305
Epoch 3/1000 | Loss: 1635.3681
Epoch 4/1000 | Loss: 1721.9843
Epoch 5/1000 | Loss: 1661.2125
Epoch 6/1000 | Loss: 1651.2715
Epoch 7/1000 | Loss: 1610.3591
Epoch 8/1000 | Loss: 1671.1390
Epoch 9/1000 | Loss: 1651.9471
Epoch 10/1000 | Loss: 1664.6019
Epoch 11/1000 | Loss: 1668.7504
Epoch 12/1000 | Loss: 1647.5927
Epoch 13/1000 | Loss: 1719.2938
Epoch 14/1000 | Loss: 1670.7513
Epoch 15/1000 | Loss: 1618.1702
Epoch 16/1000 | Loss: 1716.1271
Epoch 17/1000 | Loss: 1635.3139
Epoch 18/1000 | Loss: 1606.5372
Epoch 19/1000 | Loss: 1653.0999
Epoch 20/1000 | Loss: 1632.8381
Epoch 21/1000 | Loss: 1626.5073
Epoch 22/1000 | Loss: 1697.7222
Epoch 23/1000 | Loss: 1665.7795
Epoch 24/1000 | Loss: 1608.5107
Epoch 25/1000 | Loss: 1607.9718
Epoch 26/1000 | Loss: 1672.6042
Epoch 27/1000 | Loss: 1604.4076
Epoch 28/1000 | Loss: 1666.4660
Epoch 29/1000 | Loss: 1677.1515
Epoch 30/1000 | Loss: 1671.5077
Epoch 31/1000 | Loss: 1696.0321
Epoch 32/1000 | L

In [ ]:
with torch.no_grad():
    for inputs, targets in testloader:
        outputs = model(inputs).cpu().numpy()
        y_true = targets.cpu().numpy()

In [ ]:
mae = mean_absolute_error(y_true, outputs)
mse = mean_squared_error(y_true, outputs)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, outputs)

In [ ]:
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

MAE: 940.2071
MSE: 68908648.0000
RMSE: 8301.1230
R²: -0.0131
